# Exercise: Creating the Bob Initial Commitment Transaction

In this exercise, you'll build Bob's lightning channel initial commitment transaction from scratch using Python. While Alice creates her own commitment transaction (showing her view of the channel state), Bob also needs his own commitment transaction that represents his view. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we'll use the basepoints derivated in the `chapter 0 - lightning node keys derivation`, and the funding transaction created in the `chapter 1 - channel funding transaction`.

In [ ]:
%run "../chapter 1 - channel funding transaction/funding transaction.ipynb"

In [ ]:
from functions.test_framework.script import Tapbranch, TapTree, TapLeaf, CScript, TaprootSignatureHash, OP_CHECKSIG, OP_CHECKSIGVERIFY, OP_CHECKSEQUENCEVERIFY, OP_DROP, OP_16, OP_1, OP_SIZE, OP_EQUALVERIFY, OP_HASH160, OP_CHECKLOCKTIMEVERIFY
from functions.test_framework.messages import CTxInWitness, ser_string
TAPSCRIPT_VER = bytes([0xc0])
# A Taproot NUMS key is a fixed secp256k1 public key with no known private key. Used as the internal key to disable key-path spending, forcing script-path-only spends.
NUMS = ECPubKey().set(bytes.fromhex("02dca094751109d0bd055d03565874e8276dd53e926b44e3bd1bb6bf4bc130a279"))

At this point, no HTLCs have been added yet, which makes the initial commitment transaction simpler. Bob needs his own commitment transaction before the channel can be considered open. This serves as a guarantee for Bob that he can enforce the channel state on-chain if needed.

### The Unsigned Transaction

### The Input

The input is the channel funding transaction.

In [ ]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number on the opening channel 
commitment_number = 0
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("Commitment Number Osbscured", hex(commitment_number_obscured))

### The Outputs

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

Because Bob's balance is 0 (below dust limit), his commitment transaction will only have:

* to_remote_anchor_output (330 sats)
* to_remote_output (Alice's balance)

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) specifies that transaction outputs must be sorted by value, from smallest to largest. The to_remote_anchor_output appears first with 330 sats.

### Bob First Commitment Transaction Outputs

#### The Remote Anchor Output

This is the anchor output for the remote party (Alice). It uses Alice's payment pubkey.

```
    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^  
                   |   
         +---------------+
         | P(remote) + T |
         +---------------+
                ^  
                |  
          +-----------+
          | T = t * G |
          +-----------+     
                ^                          
                |
              +---+   +---------------------------------------------------+
              | t | = | TaggedHash ("Taptweak", P(remote) || script_root) |
              +---+   +---------------------------------------------------+
                                                                  ^  
                                                                  |
                                                                +---+
                                                                | h |
                                                                +---+
                                                                  ^  
                                                                  |
                                                            +--------------+
                                                            | OP_16 OP_CSV |
                                                            +--------------+
```

#### To Remote Output

The to_remote output pays Alice (the remote party from Bob's perspective). This output uses NUMS as internal key with a script path:
- **Internal key**: NUMS point (disables key-path spending)
- **Script path**: P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP
- **1-block CSV delay**: Alice must wait 1 block before spending
- **Not revocable**: This output remains valid even if the commitment is revoked

```
    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^  
                   |   
             +----------+
             | NUMS + T |
             +----------+
                   ^  
                   |  
              +-----------+
              | T = t * G |
              +-----------+     
                         ^                          
                         |
                       +---+   +----------------------------------------------+
                       | t | = | TaggedHash ("Taptweak", NUMS || script_root) |
                       +---+   +----------------------------------------------+
                                                                     ^  
                                                                     |
                                          +-------------------------------------------+
                                          | P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP |
                                          +-------------------------------------------+
```

#### Key Derivations

For Bob's commitment transaction, we need:
- `alice_payment_pubkey`: Alice's payment key (for remote anchor and to_remote output)

Each commitment transaction uses unique keys derived by adding a per-commitment point to their respective base points. As defined in Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G
```

In [ ]:
# TODO: Create Bob per-commitment using per_commitment function with bob_per_commitment_seed and commitment_number

# TODO: Create Alice per-commitment using per_commitment function with alice_per_commitment_seed and commitment_number

# TODO: Create Alice Payment Public Key (for remote anchor and to_remote outputs)
# Use derivate_key with alice_node_seed, family=5 (payment basepoint), channel_index=0
# Then call get_pubkey with bob_per_commitment.get_pub()

print(f"Alice Payment PubKey: {alice_payment_pubkey.get_bytes(bip340=True).hex()}")

# Outputs for Bob First Commitment Transaction
# 0x02 outputs
output_count = bytes.fromhex("02")

# REMOTE ANCHOR OUTPUT (for Alice)
# Anchor output amount
anchor_output_value_satoshis = 330
anchor_output_value = anchor_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# TODO: Create the anchor script: OP_16 OP_CSV using CScript

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# TODO: Compute Anchor Output script_root using tagged_hash with "TapLeaf" and hash_input

# TODO: Compute Anchor Output taptweak using tagged_hash
# Use "TapTweak", alice_payment_pubkey.get_bytes() + script_root
# (Note: uses Alice's payment pubkey - P(remote))

# TODO: Tweak alice_payment_pubkey with taptweak using tweak_add method

# TODO: Create scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
# remote_anchor_spk = bytes.fromhex("51") + varint_len(...) + ...


### Fee Calculation

The fee calculation for commitment transactions is based on the current `feerate_per_kw` and the **expected** weight of the transaction.

In [ ]:
# TODO: Calculate Bob's first commitment expected weight
# 0 to_local outputs (Bob has no balance), 1 to_remote output (uses script path), 1 anchor
# Note: to_remote uses NUMS + script path, so it's counted as TapOut
# Use taproot_commit_weight with NumTapOut=1, NumAnchorOut=1

print("Bob's First Commitment Transaction Expected Weight:", commitment_weight)

# From open_channel message (1 sat/vb = 250 sat/kw)
feerate_per_kw = 250

# TODO: Calculate the fee for the first commitment transaction
# commitment_fee = int(commitment_weight * feerate_per_kw / 1000)

print(f"Commitment Fee: {commitment_fee} sats")

In [ ]:
# TO REMOTE OUTPUT (Alice's balance from Bob's perspective)
# TODO: Calculate to_remote_value_sat = channel_value_sat - anchor - fee

to_remote_value = to_remote_value_sat.to_bytes(8, byteorder="little", signed=False)

# For to_remote output, we use NUMS as internal key with a script path
# Script: P(remote_payment) OP_CHECKSIG OP_1 OP_CSV OP_DROP
# This ensures Alice must wait 1 block before spending (CSV delay)

# TODO: Create the script using CScript with:
# alice_payment_pubkey.get_bytes(), OP_CHECKSIG, OP_1, OP_CHECKSEQUENCEVERIFY, OP_DROP

# TODO: Compute the script_root
# hash_input = TAPSCRIPT_VER + ser_string(script)
# script_root = tagged_hash("TapLeaf", hash_input)

# TODO: Compute TapTweak using NUMS as internal key
# taptweak = tagged_hash("TapTweak", NUMS.get_bytes() + script_root)

# TODO: Tweak NUMS with taptweak
# NUMS_tweaked = NUMS.tweak_add(taptweak)

# TODO: Create scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
# to_remote_spk = bytes.fromhex("51") + varint_len(...) + ...

outputs = (
    anchor_output_value
    + varint_len(remote_anchor_spk)
    + remote_anchor_spk
    + to_remote_value
    + varint_len(to_remote_spk)
    + to_remote_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')
locktime = locktime[::-1]

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

## The sighash for the key path spend

This is the "Schnorr key spend" case: proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [ ]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()

## Signing the sighash

In [ ]:
# Build participants to sort them using the same rule used in pubkeys aggregation.
participants = [
    (pubkey_alice_musig2.get_bytes(bip340=False), privkey_alice_musig2.get_bytes(), alice_per_commitment_seed),
    (pubkey_bob_musig2.get_bytes(bip340=False),   privkey_bob_musig2.get_bytes(), bob_per_commitment_seed),
]

# Reorder participants to match the sorted pubkeys
pk_to_tuple = {pk: (pk, sk, pcs) for pk, sk, pcs in participants}
participants = [pk_to_tuple[pk] for pk in sorted_pubkeys]

# TODO: Alice and Bob generate their own nonce pairs (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # Use per-commitment nonce for Bob to get deterministic results
    # Use nonce_gen for Alice (jit nonce)
    if pk == pubkey_bob_musig2.get_bytes(bip340=False):
        # TODO: Use nonce_per_commitment for Bob
        sec, pub = None, None  # Replace with actual call
    else:
        # TODO: Use nonce_gen for Alice
        sec, pub = None, None  # Replace with actual call
    secnonces.append(sec)
    pubnonces.append(pub)

# TODO: Aggregate the pubnonces using nonce_agg

# TODO: Create the session context using SessionContext
# session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# TODO: Alice and Bob produce their partial signatures using sign function
# psigs = [sign(sec, sk, session) for sec, (_, sk, _) in zip(secnonces, participants)]

# Each side verifies the other's partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# TODO: Combine partial signatures into the final Schnorr signature using partial_sig_agg

# Sanity check: verify with BIP340 against the *tweaked* x-only key
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)

## The signed transaction

In [ ]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the signed transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_tx.hex()]))